<a href="https://colab.research.google.com/github/dmitriygorlov/Yandex.Practikum_Data_Science/blob/main/Module-02_01-Introduction-to-machine-learning/project-5_users-behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендация тарифов

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

Импортировали библиотеки, а также записали исходную таблицу в df. изучим данные

In [3]:
df.info()

display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Таблица исходных данных содержит информацию о пользователе и его тарифе с количеством использованных им звонков, минут, сообщений и мегабайт. Все столбцы данных по пользователею с плавающей запятой, параметр тарифа - целое число. Пустых значений нет, изменения в типе данных также не нужны. Проверим целевой признак is_ultra на значения

In [4]:
print(df['is_ultra'].value_counts())

0    2229
1     985
Name: is_ultra, dtype: int64


В наборе данных по столбцу is_ultra, отвечающему за тариф, только 2 значения (1 - Ultra, 0 - Smart). Примерно две трети - это тарифы Smart, треть - Ultra. Для предсказания по тарифам, нам потребуется задача бинарной классификации, обучение с учителем.

## Разбейте данные на выборки

In [5]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

Делем исходные данные на целевой признак и признаки по которым будем предсказывать. Теперь поделим эти таблицы на обучающующую, валидационную и тестовую выборку (3:1:1) дважды применив деление

In [6]:
print('Объектов всего:', len(features))

main_features, test_features, main_target, test_target = train_test_split(
        features, target, test_size=0.2, random_state=231)

train_features, valid_features, train_target, valid_target = train_test_split(
        main_features, main_target, test_size=0.25, random_state=231)

print('Обучающих объекты:', len(train_features))
print('Валидирующие объекты:', len(valid_features))
print('Тестовые объекты:', len(test_features))


Объектов всего: 3214
Обучающих объекты: 1928
Валидирующие объекты: 643
Тестовые объекты: 643


Деление на выборки провели, теперь можно переходить к исследованию моделей

## Исследуйте модели

### Исследуем модели дерева решений

In [7]:
tree_best_model = None
tree_best_accuracy = 0
tree_best_max_depth = 0

for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=231, max_depth=depth)
    model.fit(train_features, train_target)
    valid_predictions = model.predict(valid_features)
    accuracy = accuracy_score(valid_target, valid_predictions)
    print('max_depth =', depth, ', accuracy = {:.4f}'.format(accuracy))
    if accuracy > tree_best_accuracy:
        tree_best_model = model
        tree_best_accuracy = accuracy
        tree_best_max_depth = depth
print()
print('Лучшая максимальная глубина:', tree_best_max_depth, 'Точность правильных ответов: {:.4f}'.format(tree_best_accuracy))

max_depth = 1 , accuracy = 0.7309
max_depth = 2 , accuracy = 0.7574
max_depth = 3 , accuracy = 0.7729
max_depth = 4 , accuracy = 0.7714
max_depth = 5 , accuracy = 0.7823
max_depth = 6 , accuracy = 0.7605
max_depth = 7 , accuracy = 0.7807
max_depth = 8 , accuracy = 0.7745
max_depth = 9 , accuracy = 0.7683
max_depth = 10 , accuracy = 0.7652
max_depth = 11 , accuracy = 0.7589
max_depth = 12 , accuracy = 0.7543
max_depth = 13 , accuracy = 0.7589
max_depth = 14 , accuracy = 0.7481
max_depth = 15 , accuracy = 0.7512

Лучшая максимальная глубина: 5 Точность правильных ответов: 0.7823


Наибольшей точности модель достигла с максимальной глубиной дерева 5 (близкое значение было при глубине в 7, дальше только уменьшалась точность), точность правильных ответов составила: 0,7823

### Исследуем модели случайного леса

In [8]:
forest_best_model = None
forest_best_accuracy = 0
forest_best_n_estimators = 0

for est in range(1, 41):
    model = RandomForestClassifier(random_state=231, n_estimators=est)
    model.fit(train_features, train_target)
    valid_predictions = model.predict(valid_features)
    accuracy = accuracy_score(valid_target, valid_predictions)
    print('n_estimators =', est, ', accuracy = {:.4f}'.format(accuracy))
    if accuracy > forest_best_accuracy:
        forest_best_model = model
        forest_best_accuracy = accuracy
        forest_best_n_estimators = est
print()
print('Лучшее количество деревьев:', forest_best_n_estimators, 'Точность правильных ответов: {:.4f}'.format(forest_best_accuracy))

n_estimators = 1 , accuracy = 0.7387
n_estimators = 2 , accuracy = 0.7434
n_estimators = 3 , accuracy = 0.7543
n_estimators = 4 , accuracy = 0.7558
n_estimators = 5 , accuracy = 0.7543
n_estimators = 6 , accuracy = 0.7729
n_estimators = 7 , accuracy = 0.7589
n_estimators = 8 , accuracy = 0.7745
n_estimators = 9 , accuracy = 0.7760
n_estimators = 10 , accuracy = 0.7823
n_estimators = 11 , accuracy = 0.7792
n_estimators = 12 , accuracy = 0.7807
n_estimators = 13 , accuracy = 0.7683
n_estimators = 14 , accuracy = 0.7838
n_estimators = 15 , accuracy = 0.7823
n_estimators = 16 , accuracy = 0.7807
n_estimators = 17 , accuracy = 0.7823
n_estimators = 18 , accuracy = 0.7760
n_estimators = 19 , accuracy = 0.7776
n_estimators = 20 , accuracy = 0.7838
n_estimators = 21 , accuracy = 0.7823
n_estimators = 22 , accuracy = 0.7776
n_estimators = 23 , accuracy = 0.7745
n_estimators = 24 , accuracy = 0.7854
n_estimators = 25 , accuracy = 0.7838
n_estimators = 26 , accuracy = 0.7869
n_estimators = 27 , a

Наибольшей точности модель достигла с количеством деревье 30 (хотя при 10 деревьях точность уже была близкой 0,7823), точность правильных ответов составила: 0,7885

### Исследуем модель логистической регрессии

In [9]:
log_model = LogisticRegression(random_state=231)
log_model.fit(train_features, train_target)
log_accuracy = log_model.score(valid_features, valid_target)

print('Точность правильных ответов: {:.4f}'.format(log_accuracy))

Точность правильных ответов: 0.6781


Модель логистической регрессии достигла точности правильных ответов 0,6796

**Итого наибольшей точности достигла модель случайного леса при количестве дереьев 30**. Однако и дерево решений с глубиной в 7 достигло хороших результатов (0,7823). Модель логистической регрессии не преодолела требуемый порог точности в 0.75

## Проверьте модель на тестовой выборке

In [10]:
print('Точность лучшей модели дерева решений на тестовой выборке: {:.4f}'.format(tree_best_model.score(test_features, test_target)))
print('Точность лучшей модели случайного леса на тестовой выборке: {:.4f}'.format(forest_best_model.score(test_features, test_target)))
print('Точность модели логистической регрессии на тестовой выборке: {:.4f}'.format(log_model.score(test_features, test_target)))

Точность лучшей модели дерева решений на тестовой выборке: 0.7900
Точность лучшей модели случайного леса на тестовой выборке: 0.7916
Точность модели логистической регрессии на тестовой выборке: 0.7076


Как и ожидалось лучше всего на тестовой выборке сработала лучшая модель случайного леса (30 деревьев): 0,7916, однако почти также сработала и лучшая модель дерева решений (глубина 5): 0,7900. Что, при учёте тяжелости рассчётов модели случайного леса, говорит нам о смысле **лучше всего использовать лучшую модель дерева решений**

## (бонус) Проверьте модели на адекватность

In [11]:
print('Доля тарифа Smart: ', len(df[df['is_ultra'] == 0])/len(df))

Доля тарифа Smart:  0.693528313627878


Точность выбранными нами моделями около 0,79. Количество объектов со значением Smart - 0,69. Когда мы предсказываем, мы делаем это точнее, чем если бы все объекты поставили бы Smart, значит модель адекватна!

In [13]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_features, train_target)

dummy = dummy_clf.score(test_features, test_target)
print('Доля тарифа Smart: ', dummy)

Доля тарифа Smart:  0.6982892690513219
